In [1]:
!pip3 install kaggle


[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from kaggle.api.kaggle_api_extended import KaggleApi

# Initialize the Kaggle API
api = KaggleApi()

# Authenticate using the kaggle.json file
api.authenticate()  

# Download the dataset
dataset_name = 'aungpyaeap/supermarket-sales'  # Replace with the dataset identifier
download_path = r'F:\supermarket_sales'  # Specify the folder where you want to download the dataset

api.dataset_download_files(dataset_name, path=download_path, unzip=True) 

Dataset URL: https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales


In [3]:
import pandas as pd
import sqlite3

# Load CSV file
df = pd.read_csv('F:/supermarket_sales/supermarket_sales - Sheet1.csv')

# Rename columns to match SQLite naming conventions (lowercase and replace spaces with underscores)
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Convert 'date' column from MM/DD/YYYY to YYYY-MM-DD
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

# Establish SQLite connection
conn = sqlite3.connect('sales.db')
cursor = conn.cursor()

# 1. Transforming the Branch Data (unique branch, city combinations)
branch_df = df[['branch', 'city']].drop_duplicates()
branch_df['branch_id'] = range(1, len(branch_df) + 1)  # Automatically creating the branch_id
branch_df = branch_df[['branch_id', 'branch', 'city']]

# 2. Transforming the Customer Data (unique customer_type, gender combinations)
customer_df = df[['customer_type', 'gender']].drop_duplicates()
customer_df['customer_id'] = range(1, len(customer_df) + 1)  # Automatically creating the customer_id
customer_df = customer_df[['customer_id', 'customer_type', 'gender']]

# 3. Transforming the Product Data (unique product_line, unit_price, cogs combinations)
product_df = df[['product_line', 'unit_price', 'cogs']].drop_duplicates()
product_df['product_id'] = range(1, len(product_df) + 1)  # Automatically creating the product_id
product_df = product_df[['product_id', 'product_line', 'unit_price', 'cogs']]

# 4. Insert data into the Branch Dimension
branch_df.to_sql('branch', conn, if_exists='replace', index=False)

# 5. Insert data into the Customer Dimension
customer_df.to_sql('customer', conn, if_exists='replace', index=False)

# 6. Insert data into the Product Dimension
product_df.to_sql('product', conn, if_exists='replace', index=False)

# 7. Mapping customer, product, and branch ids from the dimensions into the sales data
df = df.merge(branch_df, on=['branch','city'], how='left')
df = df.merge(customer_df, on=['customer_type', 'gender'], how='left')
df = df.merge(product_df, on=['product_line', 'unit_price', 'cogs'], how='left')

# 8. Selecting required columns for the Sales Fact Table
sales_data = df[['invoice_id', 'branch_id', 'customer_id', 'product_id', 'quantity', 'total', 'gross_income', 'date', 'time', 'payment', 'rating']]

# 9. Insert data into the Sales Fact Table
sales_data.to_sql('sales', conn, if_exists='replace', index=False)

# Commit changes and close connection
conn.commit()
conn.close()

In [4]:
# Establish SQLite connection
conn = sqlite3.connect('sales.db')
cursor = conn.cursor()

# Load SQL query data into data frame
export_df = pd.read_sql("""WITH CTE AS(
SELECT 
    b.branch,
    p.product_line,
    ROUND(SUM(s.total), 2) AS total_sales,
    RANK() OVER (PARTITION BY b.branch ORDER BY ROUND(SUM(s.total), 2) DESC) AS r
FROM 
    sales s
JOIN 
    branch b ON s.branch_id = b.branch_id
JOIN 
    product p ON s.product_id = p.product_id
GROUP BY 
    b.branch, p.product_line
)
SELECT branch,
	   product_line,
	   total_sales 
FROM CTE 
WHERE r <= 3;
""", conn)

# Export data frame data to CSV file
export_df.to_csv("report.csv",index=False)

# Commit changes and close connection
conn.commit()
conn.close()